Shopping Log - Activity Report
https://plum.greenehillfood.coop/civicrm/report/instance/103?reset=1&output=criteria

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import datetime
import itertools
from datetimerange import DateTimeRange
import pickle
import sqlalchemy

In [2]:
#os.chdir('/home/candela/Documents/greeneHill/membershipReportsCIVI')
os.chdir('/home/candela/Documents/greeneHill/membershipReportsCIVI/member_shopping')
#/home/candela/Documents/greeneHill/membershipReportsCIVI/membershipReportingLogicSampleReports
print(os.getcwd())

/home/candela/Documents/greeneHill/membershipReportsCIVI/member_shopping


In [3]:
os.listdir()

['front_desk_logs', 'civi_member_shopping']

In [4]:
shop = pd.read_csv('./civi_member_shopping/Report_20230730-1232.csv')

In [5]:
#rename fields
shop.columns = [i.replace(' ','_') for i in shop.columns]

In [6]:
shop['Activity_Date'] = pd.to_datetime(shop['Activity_Date'], format = "%Y-%m-%d %H:%M")

In [7]:
shop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8149 entries, 0 to 8148
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Assignee_Name    0 non-null      float64       
 1   Target_Name      8149 non-null   object        
 2   Source_Email     0 non-null      float64       
 3   Assignee_Email   0 non-null      float64       
 4   Target_Email     8149 non-null   object        
 5   Activity_Type    8149 non-null   object        
 6   Subject          8149 non-null   object        
 7   Activity_Date    8149 non-null   datetime64[ns]
 8   Activity_Status  8149 non-null   object        
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 573.1+ KB


Review some characteristics of the dataset

In [14]:
print(shop['Activity_Type'].value_counts())
print(shop['Activity_Status'].value_counts())
print(shop['Subject'].value_counts())
#shop['Activity Type'].value_counts()

Shop    7422
Name: Activity_Type, dtype: int64
Completed    7422
Name: Activity_Status, dtype: int64
Checked in to shop via email      5604
Checked in to shop via barcode    1818
Name: Subject, dtype: int64


Fields of interest: Target_Name, Target_Email, Activity_Type, Subject, Activity_Date

In [9]:
shop['ingest_date'] = datetime.datetime.today()

In [10]:
#establish connection to db and import the table
user = 'root'
password = 'baeldung'
host = '172.17.0.2'
port = 3306
database = 'membership'

url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(user, password, host, port, database)

engine = sqlalchemy.create_engine(url)
#, connect_args={'options': '-csearch_path=schema_name'}, isolation_level="AUTOCOMMIT"

with engine.connect() as conn:
    shop.loc[:,['Target_Name', 'Target_Email', 'Activity_Type', 'Subject', 'Activity_Date','ingest_date']].to_sql('shop_log', con=engine, if_exists='replace', index=False)